GPT = Generative Pre-Trained Transformer

In [1]:
# Read the data into a string
with open("tinyshakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
chars = sorted(list(set(text)))
data_size, vocab_size = len(text), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 1115394 characters, 65 unique.


In [3]:
# Tokenize the text into words
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

# Convert the text into a sequence of integers
encoder = lambda s: [stoi[c] for c in s]
decoder = lambda s: ''.join([itos[c] for c in s])

In [5]:
import torch

data = torch.tensor(encoder(text), dtype=torch.int64)
print(data.shape, data.dtype)
print(data[:100])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:

# Split the data into train and test
n_train = int(0.9 * data_size)
train_data = data[:n_train]
test_data = data[n_train:]

We're not going to feed the entire sequence into the model at once. \
Instead, we'll feed it a small chunk at a time, and then let it continue predicting the next character.

In [7]:

# Sometimes called the chunk size or batch size.
block_size = 8

# We will actually separate block_size + 1 characters at a time. 
# Look at it this way:

# For block[0],    we will predict block[1], 
# For block[0, 1], we will predict block[2], 
# and so on. 

# Which means we need to have block[8] as target for block[0, ... , 7]

# This is not just done for efficiency, but also to make sure that the 
# Transformer gets used to seeing contexts as little as 1 character long 
# and as long as block_size c haracters long.

In [12]:
x = train_data[ : block_size]
y = train_data[1: block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context.tolist()}, the target is {target}.")

When input is [18], the target is 47.
When input is [18, 47], the target is 56.
When input is [18, 47, 56], the target is 57.
When input is [18, 47, 56, 57], the target is 58.
When input is [18, 47, 56, 57, 58], the target is 1.
When input is [18, 47, 56, 57, 58, 1], the target is 15.
When input is [18, 47, 56, 57, 58, 1, 15], the target is 47.
When input is [18, 47, 56, 57, 58, 1, 15, 47], the target is 58.


In [ ]:
# There's one more thing to care about and that's the batch dimension.
# We want to be able to feed in a batch of sequences at a time (as a tensor) to the GPU
# and have it process all of them in parallel for a speedup.

In [9]:
# Setting a seed to make sure the same pseudo-random sequence is generated every time we run this cell
torch.manual_seed(1337)
batch_size = 4      # How many independent sequences to process in parallel
block_size = 8      # Maximum context length for predictions


def get_batch(split: str):
    # generate a small batch of data of inputs `x` and targets `y`
    data = train_data if split == "train" else test_data
    # stochastic sampling of the data, ix = indices
    ix = torch.randint(len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])

    return x, y

In [17]:
torch.manual_seed(1337)

# Print an example batch
xb, yb = get_batch("train")

print("inputs:")
print(xb.shape)
print(xb)

print("targets:")
print(yb.shape)
print(yb)

print("---")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()}, the target is {target}.")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
When input is [24], the target is 43.
When input is [24, 43], the target is 58.
When input is [24, 43, 58], the target is 5.
When input is [24, 43, 58, 5], the target is 57.
When input is [24, 43, 58, 5, 57], the target is 1.
When input is [24, 43, 58, 5, 57, 1], the target is 46.
When input is [24, 43, 58, 5, 57, 1, 46], the target is 43.
When input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39.
When input is [44], the target is 53.
When input is [44, 53], the target is 56.
When input is [44, 53, 56], the target is 1.
When input is [44, 53, 56, 1], the target is 58.
When input is [44, 53, 

A Logit function, also known as the log-odds function, is a function that represents probability values from 0 to 1, and negative infinity to infinity. The function is an inverse to the sigmoid function that limits values between 0 and 1 across the Y-axis, rather than the X-axis.

$$
logit(p) = \log(\frac{p}{1-p})
$$

In [18]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [27]:
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int) -> None:
        super().__init__()

        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx: torch.tensor, targets: torch.tensor = None) -> torch.tensor:

        # idx and targets are both (Batch, Time) tensors of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)

        # When we pass our input through the embedding table, 
        # every single integer in our input is going to refer to this embedding table
        # is going to pluck out a row from this embedding table corresponding to that integer (as an index)
        # In this case, Batch = 4, Time = 8, Channel = 65 (vocab_size)

        # Logits are the scores for the next character in the sequence
        # As you see, this is a bi-gram, which means that the next character is predicted
        # Only based the current character, and not the entire sequence

        # Aka negative log likelihood loss
        # loss = F.cross_entropy(logits, targets)

        # The above code won't work, because we have a tensor in the shape of (B, T, C)
        # But Pytorch's cross entropy expects a tensor of shape (B * T, C)
        # So basically we need to flatten the input tensor and the target tensor into a 1D tensor

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            # We will reshape the logits instead of the input here.
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
        
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: torch.tensor, max_new_tokens: int):
        # idx is a (Batch, Time) array of indices in the current context

        # The point of this is to start with a sequence and 
        # keep adding new tokens to it, and then keep feeding it back into the model
        # to get new predictions until we have the desired number of new tokens

        for _ in range(max_new_tokens):
            
            # This function, even though general, is a bit ridiculous
            # Because this is a bi-gram model, we only need to pass the last token
            # to our table, but we are passing the entire sequence to the table
            # and then, at the next step (`logits = ...`) we pick the last token
            # Out of all the tokens in the sequence

            # Later, we will use the entire sequence to predict the next token
             
            # get the predictions, we don't care about the loss because we're not training
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (Batch, Channel)
            # apply softmax to convert to probabilities
            probs = F.softmax(logits, dim=-1) # becomes (Batch, Channel)
            # sample from the distribution or take the most likely
            idx_next = torch.multinomial(probs, num_samples=1) # becomes (Batch, 1)
            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=-1) # becomes (Batch, Time + 1)
        
        return idx

In [28]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5262, grad_fn=<NllLossBackward0>)


In [ ]:
# Now, because initally our model should equally predict any character as the next character
# Our log likelihood loss should be around -ln(1/65) = 4.174387269895637

In [29]:
# We set our initial token to be 0, which is the newline character in our vocabulary

idx = torch.zeros((1, 1), dtype=torch.long)

# We have one batch and the [0] is picking the first batch
print(decoder(m.generate(idx, max_new_tokens=100)[0].tolist()))

# The model is obviously not trained so it's just spitting out random characters


'JgC.JZWqUkpdtkSpmzjM-,RqzgaN?vC:hgjnAnBZDga-APqGUH!WdCbIb;$DefOYbEvcaKGMmnO'q$KdS-'ZH
.YSqr'X!Q! d;


In [30]:
# create a PyTorch optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [34]:
batch_size = 32
max_steps = 10000

for steps in range(max_steps):
    # get a batch of data
    xb, yb = get_batch("train")

    # get the model predictions and loss
    logits, loss = m(xb, yb)

    # zero out the gradients
    optimizer.zero_grad(set_to_none=True)
    
    # compute the gradients
    loss.backward()

    # update the model parameters
    optimizer.step()

    if steps % 1000 == 0:
        print(f"Step {steps}, loss = {loss.item()}")

Step 0, loss = 3.6967320442199707
Step 1000, loss = 3.0273618698120117
Step 2000, loss = 2.821552276611328
Step 3000, loss = 2.547393798828125
Step 4000, loss = 2.519805908203125
Step 5000, loss = 2.529714584350586
Step 6000, loss = 2.542541980743408
Step 7000, loss = 2.5140950679779053
Step 8000, loss = 2.5017194747924805
Step 9000, loss = 2.5214755535125732


In [36]:
# We set our initial token to be 0, which is the newline character in our vocabulary

idx = torch.zeros((1, 1), dtype=torch.long)

# We have one batch and the [0] is picking the first batch
print(decoder(m.generate(idx, max_new_tokens=300)[0].tolist()))


CKELOresm, bur stthakls,
Ther layo-then ha nleincede jahe
DZW:
Gothe s kendwepive.
FAnorereroldghmig ppu:
Co nlllinger hus;
aver his, t towis t s ng,
ANE: foratreaisplblthriat, otimust hiny ille, yomeON p, IN I ckist vemo th.
Dieathy al hi?
Fo'd ha s?
ARS:
Semnd thinghy.
IORDitwint sth! mine actwis 


In [ ]:
# This is obviously not shakespeare, but it's a lot better than random characters!

# Stop for now at Chapter (Port our code to a script) (38:00)